In [43]:
#import libraries and set up display settings of notebook
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None  # default='warn'
import datetime
import glob
import os
import json

In [44]:
SCRIPT_PATH = os.path.abspath('')
#SCRIPT_PATH = os.path.dirname(os.path.abspath(__file__))
#LPG = os.path.join(SCRIPT_PATH, '..', 'resources', 'LPG.txt')
LPG = os.path.join(SCRIPT_PATH, 'LPG.txt')
with open(LPG, 'r') as file:
    DATA = json.load(file)
DATA['units']
AL_LOGO = os.path.join(SCRIPT_PATH, 'ALlogo.png')

In [45]:
foldername = "logs/ProcessLogs/"
# Define files path
Path = "ProcessLog?*.csv"

# Read all the files name in the directory
AllFilesname = glob.glob(foldername + Path)

# Create an empty list to store the dataframes
ListDataframe = list()

# Total file number
NoFiles = len(AllFilesname)

# For each file in the specified directory import the data and add it in the list
for Filename in AllFilesname:
    ListDataframe.append(pd.read_csv(Filename, sep=';', skiprows=3, decimal=',', encoding='unicode_escape'))

# Concatenate the files in the list in unique dataframe
DataImport = pd.concat(ListDataframe, axis=0, ignore_index=True)

# Change datatypes
DataImport['Timestamp'] = pd.to_datetime(DataImport['Timestamp'], format='%Y %m %d %H:%M:%S:%f')
# ordering ascending
DataImport = DataImport.sort_values(by='Timestamp', ascending=True)

# IMO_logs.drop_duplicates(keep=False, inplace=True)
DataImport['Timestamp'] = DataImport['Timestamp']

print (DataImport.shape)
DataImport.tail()

(482098, 171)


,Timestamp,Pressure setpoint from the ECR XC6005,Actual load of the engine XC6028,SF supply pressure PT6017,Internal temperature INT_TEMP,Power supply failure 24VDC PS_24V_FLT,Emergency stop feedback ESD_FB,External permissive A EXT_CONSENS_A,External permissive B EXT_CONSENS_B,SF supply run/stop command XC6001,440V Power supply 1 failure PS_440V_L1_FLT,Main sequence,Initialization sub sequence,Local startup sub sequence,Local stopping sub sequence,Production sub sequence,Startup sub sequence,Stopping sub sequence,Avoid RT overpressure sequence,RT emptying sequence,P1001A nitrogen injection sequence,P1001B nitrogen injection sequence,P1101A nitrogen injection sequence,P1101B nitrogen injection sequence,P1101A deaeration sequence,P1101B deaeration sequence,P1101A cryosump filling sequence,P1101B cryosump filling sequence,DPIT1004 - Differential pressure software transmit,TIT1003 - Temperature transmitter TIT1003,TIT1002 - Temperature transmitter TIT1002,PV1001 - Regulating valve position feedback PV1001,PIT1003 - Pressure transmitter PIT1003,PIT1004 - Pressure transmitter PIT1004,PIT1005 - Pressure transmitter PIT1005,PIT7001-1A - API plan for pump P1001A PIT7001_1A,TIT1004 - Temperature transmitter TIT1004,FIT1101 - HP minimum flow FIT1101,DPIT1102 - Differential pressure software transmit,DPIT1103 - Differential pressure software transmit,PIT1101 - Pressure transmitter PIT1101,PIT1102 - Pressure transmitter PIT1102,PIT1103 - Pressure transmitter PIT1103,TIT1102 - Temperature transmitter TIT1102,FV1101 - Regulating valve position feedback FV1101,XV1101 - Limit swicth open - Return line LPG ZS110,XV1101 - Limit swicth close - Return line LPG ZS11,TIT4004 - Temperature transmitter TIT4004,LIT1001 - Level transmitter LIT1001,TIT1006 - Temperature transmitter TIT1006,TIT1005 - Temperature transmitter TIT1005,PIT1006 - Pressure transmitter PIT1006,PIT1007 - Pressure transmitter PIT1007,PV1002 - Regulating valve position feedback PV1002,PV1003 - Regulating valve position feedback PV1003,XV1102 - Limit swicth close - Recovery tank LPG in,LS1002 - Level switch - HH recovery tank LS1002,LS1001 - Level switch - LL recovery tank LS1001,XV1001 - Limit swicth open - Recovery tank LPG out,XV1001 - Limit swicth close - Recovery tank LPG ou,XV1102 - Limit swicth open - Recovery tank LPG inl,PIT9001 - Pressure transmitter PIT9001,FIC1002 - Setpoint,FIC1002 - Process value,FIC1002 - PID Output,FIC1101 - Setpoint,FIC1101 - Process value,FIC1101 - PID Output,PIC1003 - Setpoint,PIC1003 - Process value,PIC1003 - PID Output,PIC1005 - Setpoint,PIC1005 - Process value,PIC1005 - PID Output,PIC1006_1 - Setpoint,PIC1006_1 - Process value,PIC1006_1 - PID Output,PIC1006_2 - Setpoint,PIC1006_2 - Process value,PIC1006_2 - PID Output,PIC1103 - Setpoint,PIC1103 - Process value,PIC1103 - PID Output,PIC1103 - Feedforward Value,TIC1101 - Setpoint,TIC1101 - Process value,TIC1101 - PID Output,TIC4003_1 - Setpoint,TIC4003_1 - Process value,TIC4003_1 - PID Output,TIC4003_2 - Setpoint,TIC4003_2 - Process value,TIC4003_2 - PID Output,SF supply running XC6003,SF supply ready XC6002,Common alarm signal CMN_ALARM,System light - Running LIGHT_RUN,System light - Ready to start LIGHT_RDY_TO_STR,System light - Common alarm LIGHT_CMN_ALARM,System light - Stop LIGHT_STP,SF Return system ready XC6053,PV1001 - Regulating valve position command PV1001,FV1101 - Regulating valve position command FV1101,XV1101 - Valve command - LPG Return Line SV1101,PV1003 - Regulating valve position command PV1003,PV1002 - Regulating valve position command PV1002,XV1001 - Valve command - Recovery Tank LPG Outlet,XV1102 - Valve command - Recovery Tank LPG Inlet S,P1001A - LP Svane Pump - PT100 motor bearing D-END,P1001A - LP Svane Pump - PT100 motor bearing ND-EN,P1001A - LP Svane Pump - PT100 pump bearing P1001A,PIT1002 - Pressure transmitter PIT1002,P1001A - LP Svane Pump - Oil level P1001A_OIL_LVL,FIT1002A - LP minimum flow FIT1002A,P1001A - LP Pump - Speed feedback P1001A_FB_SPD,LS1005A - Level 

In [46]:
Filename = 'logs/AlarmHistory/Alarms_2023_11_06_00_32_23.csv'

alarmhist = pd.read_csv(Filename, sep=',', decimal='.', encoding='unicode_escape')

# Rename Time
alarmhist.rename(columns = {'Time':'Timestamp',' Change':'Change',
                        ' Instance':'Instance', ' Name':'Name',
                         ' Code':'Code', ' Severity':'Severity',
                         ' AdditionalInformation1':'AdditionalInformation1',
                         ' AdditionalInformation2':'AdditionalInformation2',
                         ' Message':'Message'}, inplace = True)


# Change datatypes
alarmhist['Timestamp'] = pd.to_datetime(alarmhist['Timestamp'], format='%Y-%m-%d %H:%M:%S:%f')
# ordering ascending
alarmhist = alarmhist.sort_values(by='Timestamp', ascending=True)
# translate values
alarmhist['Change'] = alarmhist['Change'].replace({' Active -> Inactive' : '0',
                                                   ' Inactive -> Active' : '1',
                                                   ' Unacknowledged -> Acknowledged': '2'})
alarmhist['Change'] = alarmhist['Change'].astype(int)

# Show results
print (alarmhist.shape)
alarmhist.head()  

(780, 9)


,Timestamp,Instance,Name,Code,Severity,AdditionalInformation1,AdditionalInformation2,Change,Message
0,2023-10-13 17:59:22.468,44,A404,404,1,,,0,A404 - DPIT1103 Differential pressure transmi...
1,2023-10-13 17:59:36.468,44,A404,404,1,,,1,A404 - DPIT1103 Differential pressure transmi...
2,2023-10-13 17:59:39.668,44,A404,404,1,,,0,A404 - DPIT1103 Differential pressure transmi...
3,2023-10-13 17:59:51.668,44,A404,404,1,,,1,A404 - DPIT1103 Differential pressure transmi...
4,2023-10-13 17:59:56.268,44,A404,404,1,,,0,A404 - DPIT1103 Differential pressure transmi...


In [47]:
def classify_cols(selected):
    print("classify_cols started ---")
    print(f"{selected=}")

    units = DATA['units']

    filter_unit_cols = {col : unit for col, unit in units.items() if col in selected}
    # {key_expression: value_expression for item in iterable if condition}
    print(f"{filter_unit_cols=}")

    classified_cols = {}
    for col, unit in filter_unit_cols.items():
        if unit not in classified_cols:
            classified_cols[unit] = [] #if first column of this unit, create key and an empty array as value
        classified_cols[unit].append(col)
    
    print(f"{classified_cols=}")
    return classified_cols

In [48]:
import pandas as pd
from PIL import Image
import datetime
import os
import json
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Alfa Laval brand colors
ALcolors = ['rgba(17, 56, 127, 1)', #AL blue
            'rgba(0, 0, 0, 1)', #AL white
            'rgba(220, 146, 118, 1)', #AL earth
            'rgba(254, 205, 96, 1)', #AL sun
            'rgba(147, 199, 198, 1)', #AL water
            'rgba(0, 127, 200, 1)', #AL innovation
            ]

In [49]:
def plot_alarms(x,y,labels,cat):
    trace = go.Scatter(x=x, y=y.astype(str),
                       name=cat,
                       mode='markers', marker_symbol='x',
                       marker_line_color=ALcolors[2], marker_color=ALcolors[3],
                       marker_line_width=1, marker_size=8,
                       legendgroup="Alarms",legendgrouptitle_text="Alarms",
                       hovertext=labels)
    return trace

def plot_events(x,y,labels):
    trace = go.Scatter(x=x, y=y.astype(int),
                       name='Event Number',
                       mode='markers',marker_symbol='star',
                       marker_line_color=ALcolors[5], marker_color=ALcolors[5],
                       marker_line_width=1, marker_size=8,
                       legendgroup="Events",legendgrouptitle_text="Events",
                       hovertext=labels)
    return trace

def line_trace(x,y,name,cat,color=None):
    if color is not None:
        trace = go.Scatter(x=x, y=y,
                           name = name,
                           line=dict(color = color), line_shape='spline',
                           legendgroup=cat, legendgrouptitle_text=cat)    
    else: #default plotly color assignment
        trace = go.Scatter(x=x, y=y,
                           name = name,
                           line_shape='spline',
                           legendgroup=cat, legendgrouptitle_text=cat)
    return trace

def square_line_trace(x,y,name,cat,color=None,labels=None):
    if color is not None:
        trace = go.Scatter(x=x, y=y,
                           name = name,
                           line=dict(color = color), line_shape='hv',
                           legendgroup=cat, legendgrouptitle_text=cat)
    else: #default plotly color assignment
        trace = go.Scatter(x=x, y=y,
                   name = name, line_shape='hv',
                   legendgroup=cat, legendgrouptitle_text=cat)

    if isinstance(labels, pd.Series):
        trace.hovertext = labels
    
    return trace

def square_disc_line_trace(x,y,name,color,cat):
    trace = go.Scatter(x=x, y=y,
                       name = name,
                       line=dict(color = color, dash='dot'), line_shape='hv',
                       legendgroup=cat, legendgrouptitle_text=cat)
    return trace

def filled_trace(x,y,name,color,cat):
    trace = go.Scatter(x=x,y=y,
                       name= name,
                       fill='tozeroy', mode='none', 
                       fillcolor = color,
                       line_shape='hv')
    if cat is not None:
        trace.legendgroup = cat
        trace.legendgrouptitle_text = cat
    
    return trace

In [50]:
def custom_plot_divided(dfs, dfa, dfe, cols, date1, date2, tittle): # n rows, one for each unit
    print("custom_plot1 started ---")


    # Columns with its respective unit
    units = DATA['units']

    # Alarms classification
    alarm_cats = DATA['alarm_cats']

    print("date limits:")
    print(f"{date1=},{date2=}")
    
    # filter dataframes for date interval selected
    if not dfs.empty:
        dfs = dfs[(dfs['DateTime'] >= date1) & (dfs['DateTime'] <= date2)]
    else:
        dfs = pd.DataFrame()
        print("Standard Logs empty in date range selected")

    if not dfa.empty:
        dfa = dfa[(dfa['DateTime'] >= date1) & (dfa['DateTime'] <= date2)]
    else:
        dfa = pd.DataFrame()
        print("Alarm Logs empty in date range selected")

    if not dfe.empty:
        dfe = dfe[(dfe['DateTime'] >= date1) & (dfe['DateTime'] <= date2)]
    else:
        dfe = pd.DataFrame()
        print("Event Logs empty in date range selected")

    #returns an empty plot if no data to plot
    if dfs.empty and dfa.empty and dfe.empty:
        fig = go.Figure()
        fig.update_layout(title_text=tittle)
        print("--- no data to plot")
        return fig

    # From column list, get a dictionary with cols classified by unit type
    cols2 = classify_cols(cols)
    print("selected units and columns:")
    print(cols2)

    print("fig init")
    fig = make_subplots(rows=len(cols2), cols=1, shared_xaxes=True, vertical_spacing=0.02)

    # Iterate classified cols and create traces
    for i, (unit, cols) in enumerate(cols2.items()):
        print(f"{i=}, {unit=}, {cols=}")

        fig.update_yaxes(title_text=unit,row=i+1)

        for col in cols:
            print(f"{col=}")

            if col == 'AlarmNumber':
                for cat, limits in alarm_cats.items():
                    print(cat)

                    filter_alm = dfa[(dfa['AlarmNumber'] >= limits[0]) & (dfa['AlarmNumber'] <= limits[1])]
                    if not filter_alm.empty:
                        trace = plot_alarms(
                            x=filter_alm['DateTime'],
                            y=filter_alm['AlarmNumber'],
                            labels=filter_alm['Label'],
                            cat=cat)
                        fig.add_trace(trace, row=i+1, col=1)

            elif col == 'EventNumber':
                if not dfe.empty:
                    trace = plot_events(
                        x=dfe['DateTime'],
                        y=dfe['EventNumber'],
                        labels=dfe['Label'])
                    
                    fig.add_trace(trace, row=i+1, col=1)

            elif col == 'MachineStatus':
                trace = square_line_trace(
                    x=dfs['DateTime'],
                    y=dfs['MachineStatus'],
                    labels=dfs['STS_Label'],
                    name='MachineStatus',
                    color=ALcolors[5],
                    cat='Events')
                    
                fig.add_trace(trace, row=i+1, col=1)

            elif 'CV' in col:
                trace = square_line_trace(
                    x=dfs['DateTime'],
                    y=dfs[col],
                    labels=dfs[col.split('_')[0]+'_Label'],
                    name=col,
                    cat=unit)
                fig.add_trace(trace, row=i+1, col=1)

            elif (col == 'ControlType') or (col == 'CurrentControl'):
                trace = square_line_trace(
                    x=dfs['DateTime'],
                    y=dfs[col],
                    labels=dfs['CC_Label'],
                    name=col,
                    cat=unit)
                fig.add_trace(trace, row=i+1, col=1)

            else:
                if not dfs.empty:
                    # If unit is a bool, int or valve_pos(int), create a square line trace instead of spline
                    if unit in ['bool', 'int', 'valve_pos']:
                        trace = square_line_trace(
                            x=dfs['DateTime'],
                            y=dfs[col],
                            name=col,
                            cat=unit)
                    else:
                        trace = line_trace(
                            x=dfs['DateTime'],
                            y=dfs[col],
                            name=col,
                            cat=unit)

                    fig.add_trace(trace, row=i+1, col=1)

    print("fig config")

    # Update layout properties
    fig.update_layout(hovermode="x unified", hoverlabel=dict(bgcolor='rgba(255,255,255,0.75)', namelength = -1, font=dict(color='black')),  
        legend=dict(groupclick="toggleitem"), #avoid grouping all traces
        title_text=tittle , title_x=0.5
    )

    # Add image
    alLogo = Image.open(AL_LOGO)
    fig.add_layout_image(
        dict(
            source=alLogo,
            xref="paper", yref="paper",
            x=0, y=1.025,
            sizex=0.14, sizey=0.14,
            xanchor="left", yanchor="bottom"
        )
    )
    
    print("fig done")
    return fig